In [1]:
! pip install monai
! pip install tifffile
! pip install bottleneck


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from monai.transforms import (
    Compose, ScaleIntensity, RandRotate90, RandAffine,
    Rand3DElastic, RandGaussianNoise, RandGaussianSmooth, RandZoom
)
from tifffile import TiffFile
import bottleneck  # Ensure required bottleneck version is installed

# Step 1: Patch MONAI's misc.py to replace np.bool with bool
misc_path = "/Users/nayeb/opt/anaconda3/lib/python3.9/site-packages/monai/utils/misc.py"
with open(misc_path, "r") as file:
    lines = file.readlines()

with open(misc_path, "w") as file:
    for line in lines:
        file.write(line.replace("np.bool", "bool"))

print("Patched misc.py to replace np.bool with bool")

# Step 2: Define transformations without spatial_size
def define_transforms():
    image_transforms = Compose([
        ScaleIntensity(),
        RandRotate90(prob=0.5, spatial_axes=(0, 1)),
        RandAffine(prob=0.5, translate_range=(5, 5, 5), rotate_range=(0.1, 0.1, 0.1), scale_range=(0.1, 0.1, 0.1)),
        Rand3DElastic(prob=0.3, sigma_range=(5, 8), magnitude_range=(100, 200)),
        RandGaussianNoise(prob=0.2, mean=0.0, std=0.1),
        RandGaussianSmooth(prob=0.2, sigma_x=(0.5, 1.0), sigma_y=(0.5, 1.0), sigma_z=(0.5, 1.0)),
        RandZoom(prob=0.3, min_zoom=0.9, max_zoom=1.1, mode="trilinear")
    ])

    segmentation_transforms = Compose([
        RandRotate90(prob=0.5, spatial_axes=(0, 1)),
        RandAffine(prob=0.5, translate_range=(5, 5, 5), rotate_range=(0.1, 0.1, 0.1), scale_range=(0.1, 0.1, 0.1)),
        Rand3DElastic(prob=0.3, sigma_range=(5, 8), magnitude_range=(100, 200)),
        RandZoom(prob=0.3, min_zoom=0.9, max_zoom=1.1, mode="trilinear")
    ])
    
    return image_transforms, segmentation_transforms

# Step 3: Load data as 3D
def load_data(image_path, seg_path):
    # Load image
    if image_path.endswith('.npy'):
        image = np.load(image_path, allow_pickle=True)
    else:
        with TiffFile(image_path) as tif:
            image = tif.asarray()
            print("Loaded image shape:", image.shape)

    # Attempt to load segmentation, fallback to zero array if not loaded
    try:
        if seg_path.endswith('.npy'):
            segmentation = np.load(seg_path, allow_pickle=True)
        else:
            with TiffFile(seg_path) as tif:
                segmentation = tif.asarray()
                print("Loaded segmentation shape:", segmentation.shape)
    except Exception as e:
        print(f"Error loading segmentation: {e}")
        segmentation = np.zeros_like(image)

    # Ensure segmentation has the same shape as image
    if segmentation.shape != image.shape:
        print("Invalid segmentation shape, using a zero array as fallback.")
        segmentation = np.zeros_like(image)

    print("Final image shape:", image.shape)
    print("Final segmentation shape:", segmentation.shape)

    return {"image": image, "segmentation": segmentation}

# Load the sample data
sample_data = load_data(
    "/Users/nayeb/Downloads/DL_MBL_Data_DP/annotated_images/image_test/TC_17_Cortical2-1-2.tiff", 
    "/Users/nayeb/Downloads/DL_MBL_Data_DP/annotated_seg/seg_test/TC_17_Cortical2-1-2_seg.npy"
)

# Define and apply transformations without spatial_size
image_transforms, segmentation_transforms = define_transforms()

# Apply transformations if valid shapes are detected
try:
    print("Applying transformations...")
    augmented_image = image_transforms(sample_data["image"])
    print(f"Augmented image shape: {augmented_image.shape}")
    augmented_segmentation = segmentation_transforms(sample_data["segmentation"])
    print(f"Augmented segmentation shape: {augmented_segmentation.shape}")

    # Visualize the augmented middle slice along the depth axis
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    depth_mid = augmented_image.shape[2] // 2  # Middle slice index
    axes[0].imshow(augmented_image[:, :, depth_mid], cmap="gray")
    axes[0].set_title("Augmented Image Slice")
    axes[1].imshow(augmented_segmentation[:, :, depth_mid], cmap="gray")
    axes[1].set_title("Augmented Segmentation Slice")
    plt.show()

except Exception as e:
    print(f"Transformation error: {e}")


In [ ]:
from tifffile import TiffWriter
import os
import numpy as np
import torch

# Define the number of augmented versions you want to save
num_versions = 20

# Create output directories if they do not exist
output_image_dir = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE"
output_seg_dir = "/Users/nayeb/Desktop/UCSF_LAB_PRACTICE"
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_seg_dir, exist_ok=True)

# Load the sample data
sample_data = load_data(
    "/Users/nayeb/Downloads/DL_MBL_Data_DP/unannotated/TC_1-1.tif", 
    "/Users/nayeb/Downloads/DL_MBL_Data_DP/annotated_seg/seg_test/TC_17_Cortical2-1-2_seg.npy"
)

# Define transformations with Rand3DElastic and RandZoom temporarily removed
def define_transforms_without_elastic_zoom():
    image_transforms = Compose([
        ScaleIntensity(),
        RandRotate90(prob=0.5, spatial_axes=(0, 1)),
        RandAffine(prob=0.5, translate_range=(5, 5, 5), rotate_range=(0.1, 0.1, 0.1), scale_range=(0.1, 0.1, 0.1)),
        RandGaussianNoise(prob=0.2, mean=0.0, std=0.1),
        RandGaussianSmooth(prob=0.2, sigma_x=(0.5, 1.0), sigma_y=(0.5, 1.0), sigma_z=(0.5, 1.0))
    ])

    segmentation_transforms = Compose([
        RandRotate90(prob=0.5, spatial_axes=(0, 1)),
        RandAffine(prob=0.5, translate_range=(5, 5, 5), rotate_range=(0.1, 0.1, 0.1), scale_range=(0.1, 0.1, 0.1)),
        RandGaussianNoise(prob=0.2, mean=0.0, std=0.1),
        RandGaussianSmooth(prob=0.2, sigma_x=(0.5, 1.0), sigma_y=(0.5, 1.0), sigma_z=(0.5, 1.0))
    ])
    
    return image_transforms, segmentation_transforms

# Use the modified transformations
image_transforms, segmentation_transforms = define_transforms_without_elastic_zoom()

# Generate and save multiple augmented versions
for i in range(num_versions):
    # Apply transformations to create augmented versions
    augmented_image = image_transforms(sample_data["image"])
    augmented_segmentation = segmentation_transforms(sample_data["segmentation"])

    # Convert to numpy if the output is a torch Tensor
    if isinstance(augmented_image, torch.Tensor):
        augmented_image = augmented_image.numpy()
    if isinstance(augmented_segmentation, torch.Tensor):
        augmented_segmentation = augmented_segmentation.numpy()

    # Define unique output paths for each version
    output_image_path = os.path.join(output_image_dir, f"TC_17_Cortical2-1-2_augmented_{i+1}.tiff")
    output_seg_path = os.path.join(output_seg_dir, f"TC_17_Cortical2-1-2_seg_augmented_{i+1}.npy")

    # Save the augmented image as a TIFF file
    with TiffWriter(output_image_path) as tif:
        tif.write(augmented_image.astype(np.float32))

    # Save the augmented segmentation as a Numpy file
    np.save(output_seg_path, augmented_segmentation)

    print(f"Saved augmented version {i+1} - Image: {output_image_path}, Segmentation: {output_seg_path}")

print("All augmented versions saved successfully.")

